In [ ]:
import os
USE_GOOGLE_DRIVE = False
MOUNT_GDIR = "/content/gdrive/MyDrive/Colab Notebooks/game_simulation"
OUTPUT_DIR_NAME = "data"

def mount_google_drive():
    from google.colab import drive
    mount_directory = "/content/gdrive"
    drive.mount(mount_directory)
    return None

if USE_GOOGLE_DRIVE:
    mount_google_drive()
    os.chdir(MOUNT_GDIR)

if not os.path.isdir(OUTPUT_DIR_NAME):
    os.mkdir(OUTPUT_DIR_NAME)
print(os.getcwd())

In [ ]:
from datetime import datetime
today = datetime.now()
timestamp_dir = OUTPUT_DIR_NAME+"/"+today.strftime('%Y%m%d%H%M%S')
if not os.path.isdir(timestamp_dir):
    os.mkdir(timestamp_dir)
else:
    print("Target directory exits!")

In [ ]:
MULTIPROCESSING = False
PARALLEL_COMPUTING = False

if MULTIPROCESSING:
    import multiprocessing as mp
    print(mp.cpu_count())

if PARALLEL_COMPUTING: 
    from ipyparallel import Client
    rc = Client(profile="default")
    dv = rc[:]  # dv = rc.load_balanced_view()
    print(rc.ids)
    with dv.sync_imports():
        import numpy as np
        from player import Player
        from mall import Mall
        from house import House
        import matplotlib.pyplot as plt
        import csv

if not PARALLEL_COMPUTING: 
    import numpy as np
    from player import Player
    from mall import Mall
    from house import House
    import matplotlib.pyplot as plt
    import csv
    
%matplotlib inline

In [ ]:
HOUSE_CHOICE_FUNCTIONS_SET = ('random', 'C_Func_1', 'C_Func_2', 'C_Func_3_v0', 'C_Func_3_v1')

In [ ]:
def game_run(total_interval, n_action, n_mall, house_action_label=None, **kwargs):
    if not house_action_label in HOUSE_CHOICE_FUNCTIONS_SET:
        raise  Exception("Wrong house choice function(s)!")
    else:
        mall_ls = [Mall() for _ in range(n_mall)]
        house = House(n_action)

        if not os.path.isdir(timestamp_dir+'/'+house_action_label):
            os.mkdir(timestamp_dir+'/'+house_action_label)

        if house.sampled_action_state:
            save_edge_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"edge_history_"+str(SAMPLE_ACTION_NUMBER)+"_sampled_from_"
                +str(n_action)+"_actions_"+str(n_mall)+"_malls"+".csv"
            )

            save_volume_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"volume_history_"+str(SAMPLE_ACTION_NUMBER)+"_sampled_from_"
                +str(n_action)+"_actions_"+str(n_mall)+"_malls"+".csv"
            )

            save_profit_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"profit_history_"+str(SAMPLE_ACTION_NUMBER)+"_sampled_from_"
                +str(n_action)+"_actions_"+str(n_mall)+"_malls"+".csv"
            )
        else:
            save_edge_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"edge_history_"+str(n_action)+"_actions_"
                +str(n_mall)+"_malls"+".csv"
            )
            save_volume_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"volume_history_"+str(n_action)+"_actions_"
                +str(n_mall)+"_malls"+".csv"
            )

            save_profit_file_name = (
                timestamp_dir+'/'+house_action_label+'/'
                +"profit_history_"+str(n_action)+"_actions_"
                +str(n_mall)+"_malls"+".csv"
            )

        csv_edge_file = open(save_edge_file_name, 'w', newline='')
        csv_volume_file = open(save_volume_file_name, 'w', newline='')
        csv_profit_file = open(save_profit_file_name, 'w', newline='')
        edge_writer = csv.writer(csv_edge_file, delimiter=',')
        volume_writer = csv.writer(csv_volume_file, delimiter=',')
        profit_writer = csv.writer(csv_profit_file, delimiter=',')
        if house_action_label == 'random':
            if kwargs:
                message = (f"Warning! choice function"
                           "'random' do not need extra"
                           " keywords like {kwargs}!")
                print(message)
            else:
                pass
        else:
            pass

        for t in range(total_interval):
            for m in mall_ls:
                m.player_betting(n_action)

            house.get_interval(t)
            if house_action_label == 'random':
                #if kwargs:
                #    print("Warning! choice function 'random' do not need extra keywords!")

                house_action = house.choice_func_random_action(mall_ls)
            elif house_action_label == 'C_Func_1':
                house_action = house.choice_func_1(
                    mall_ls, kwargs['upper_bound'], kwargs['lower_bound']
                )
            elif house_action_label == 'C_Func_2':
                house_action = house.choice_func_2(
                    mall_ls, kwargs['upper_bound'], kwargs['lower_bound'],
                    kwargs['P'], kwargs['Q']
                )
            elif house_action_label == 'C_Func_3_v0':
                house_action = house.choice_func_3(
                    mall_ls, kwargs['upper_bound'], kwargs['lower_bound']
                )
            elif house_action_label == 'C_Func_3_v1':
                house_action = house.choice_func_3_v1(
                    mall_ls, kwargs['upper_bound'], kwargs['lower_bound']
                )
            else:
                raise  Exception("Unknown error!")

            for m in mall_ls:
                m.update_for(house_action, house.random_action)
                # m.edge_record()  # This may consumes a lot of memory.
                # m.reset_manipulated()
            edge_writer.writerow([m.edge for m in mall_ls])
            volume_writer.writerow([m.volume for m in mall_ls])
            profit_writer.writerow([m.profit for m in mall_ls])

        final_edge_ls = [m.edge  for m in mall_ls]
        print(f"Final edges: {final_edge_ls}")
        csv_edge_file.close()
        csv_volume_file.close()
        csv_profit_file.close()
        if True:
            plot_edge_history(
                mall_ls,
                total_interval=total_interval,
                upper_bound=upper_bound,
                lower_bound=lower_bound,
                house_action_label=house_action_label
            )
        else:
            pass
        return None

In [ ]:
def plot_edge_history(mall_ls, total_interval, upper_bound=None, lower_bound=None, house_action_label=''):
    fig = plt.figure()
    for m in mall_ls:
        plt.plot(m.edge_history)

    if upper_bound:
        plt.plot(np.zeros(total_interval) + upper_bound, "k--")
    if lower_bound:
        plt.plot(np.zeros(total_interval) + lower_bound, "k--")

    plt.grid(True)
    plt.xlabel("intervals")
    plt.ylabel("edge")
    if house_action_label:
        plt.title(str(len(mall_ls))+' mall(s)'+' under '+ house_action_label)
    else:
        plt.title(str(len(mall_ls))+' mall(s)')

    fig.savefig(timestamp_dir+"/"+"edge_history_"+str(n_action)+"_actions_"+str(n_mall)+"_malls"+"_under_"+house_action_label+".png")
    return fig

def plot_final_edge(mall_ls, total_interval, upper_bound=None, lower_bound=None, house_action_label=''):
    fig = plt.figure()
    plt.bar([i for i in range(n_mall)], np.array([m.edge_history[-1] for m in mall_ls]), align="edge")
    #plt.bar([i+0.5 for i,_ in enumerate(mall_id)], np.array([m.edge_history[-1] for m in mall_ls]))
    plt.plot(np.zeros(n_mall+1) + upper_bound, "k--")
    plt.plot(np.zeros(n_mall+1) + lower_bound, "k--")
    plt.xlabel("mall_id")
    plt.ylabel("Final_edge")
    fig.savefig(timestamp_dir+"/"+"final_edge_"+str(n_action)+"_actions_"+str(n_mall)+"_malls"+"_under_"+house_action_label+".png")
    return fig

def plot_edge_convergence(mall_ls, total_interval, upper_bound=None, lower_bound=None, house_action_label='', method='in_out'):
    fig = plt.figure()
    if method == "in_out":
       out_target = [list(map(lambda x:x < lower_bound or x > upper_bound, m.edge_history)) for m in mall_ls]
       plt.plot(np.mean(out_target, axis=0))
       plt.ylabel("out_target_ratio")
    elif method == "abs":
       out_target_abs = [list(map(lambda x:lower_bound - x if x < lower_bound else
                                  x-upper_bound if  x > upper_bound else 0, m.edge_history))
                         for m in mall_ls
       ]
       plt.plot(np.mean(out_target_abs, axis=0))
       plt.ylabel("mean_distance_of_order_1")
    else:
        print("Something odd! Figure out this!")
        assert True == False

    plt.plot(np.zeros(total_interval), "k--")

    plt.grid(True)
    plt.xlabel("intervals")
    
    if house_action_label:
        plt.title(str(len(mall_ls))+' mall(s)'+' under '+ house_action_label)
    else:
        plt.title(str(len(mall_ls))+' mall(s)')
    fig.savefig(timestamp_dir+"/"+"convergence_"+str(n_action)+"_actions_"+str(n_mall)+"_malls"+"_under_"+house_action_label+"_"+method+".png")
    return fig

# Parameters

In [ ]:
total_interval = 5
n_mall = 2
n_action = 100

# Parameters for choice_func_1 and  choics_func_2
upper_bound = 0.05
lower_bound = 0.03
P = 1 
Q = 0.3

In [ ]:
kwargs = {"total_interval":total_interval,
          "n_action":n_action,
          "n_mall":n_mall,
          "upper_bound":upper_bound,
          "lower_bound":lower_bound}

# The outputs

In [ ]:
%%time
game_run(**kwargs, house_action_label='random')

In [ ]:
%%time
game_run(**kwargs, house_action_label='C_Func_1')

In [ ]:
%%time
game_run(**kwargs, house_action_label='C_Func_2', P=P, Q=Q)

In [ ]:
%%time
game_run(**kwargs, house_action_label='C_Func_3_v0')

In [ ]:
%%time
game_run(**kwargs, house_action_label='C_Func_3_v1')